In [18]:
import pandas as pd
import numpy as np

In [19]:
prices = pd.read_csv("prices.csv")
sales = pd.read_csv("sales.csv")

print(prices)
print(sales)

    product_id  old_price  new_price     updated_at
0           64     270000     239000  9/10/18 16:37
1      3954203      60000      64000  9/11/18 11:54
2      3954203      60500      57500  9/17/18 22:59
3      3954203      64000      60500   9/15/18 3:49
4      3954203      68800      60000  9/10/18 16:32
5      3998909      15500      16500   9/16/18 5:09
6      3998909      17000      15500   9/13/18 6:43
7      3998909      19000      17000  9/10/18 16:35
8      4085861      53500      52000  9/17/18 22:59
9      4085861      53500      67000   9/12/18 3:51
10     4085861      58000      53500   9/17/18 3:35
11     4085861      60000      53500   9/11/18 8:51
12     4085861      62500      58000   9/15/18 3:51
13     4085861      67000      62500   9/13/18 6:43
     product_id     ordered_at  quantity_ordered
0       3998909  9/18/18 17:51                 1
1       3998909  9/18/18 12:52                 1
2       3998909  9/18/18 11:33                 1
3       3998909  9/18/18

In [20]:
# convert datetime
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
sales['ordered_at'] = pd.to_datetime(sales['ordered_at'])

In [21]:
prices.dropna()
sales.dropna()

# sort theo ngày
prices = prices.sort_values(by='updated_at')
sales = sales.sort_values(by='ordered_at')

# merge 2 bảng
pns = pd.merge_asof(sales, prices, by='product_id', left_on='ordered_at', right_on='updated_at', direction='nearest')
pns.head

<bound method NDFrame.head of      product_id          ordered_at  quantity_ordered  old_price  new_price  \
0       3954203 2018-09-11 01:43:00                 1      68800      60000   
1       4085861 2018-09-11 06:26:00                 1      60000      53500   
2       4085861 2018-09-11 06:53:00                 1      60000      53500   
3       4085861 2018-09-11 08:24:00                 1      60000      53500   
4       4085861 2018-09-11 09:30:00                 1      60000      53500   
..          ...                 ...               ...        ...        ...   
170     4085861 2018-09-18 20:23:00                 1      53500      52000   
171     4085861 2018-09-18 20:43:00                 1      53500      52000   
172     4085861 2018-09-18 20:54:00                 1      53500      52000   
173     3954203 2018-09-18 21:26:00                 1      60500      57500   
174     3998909 2018-09-18 22:11:00                 1      15500      16500   

             updated_

In [22]:
# compare ordered_at and "nearest" updated_at of each order to find the correct price
#nếu ordered_at >= updated_at thì lấy new_price, otherwise lấy old_price cho cột real_price

pns['real_price'] = np.where(pns['ordered_at'] >= pns['updated_at'],
                                         pns['new_price'], pns['old_price'])
pns.head

<bound method NDFrame.head of      product_id          ordered_at  quantity_ordered  old_price  new_price  \
0       3954203 2018-09-11 01:43:00                 1      68800      60000   
1       4085861 2018-09-11 06:26:00                 1      60000      53500   
2       4085861 2018-09-11 06:53:00                 1      60000      53500   
3       4085861 2018-09-11 08:24:00                 1      60000      53500   
4       4085861 2018-09-11 09:30:00                 1      60000      53500   
..          ...                 ...               ...        ...        ...   
170     4085861 2018-09-18 20:23:00                 1      53500      52000   
171     4085861 2018-09-18 20:43:00                 1      53500      52000   
172     4085861 2018-09-18 20:54:00                 1      53500      52000   
173     3954203 2018-09-18 21:26:00                 1      60500      57500   
174     3998909 2018-09-18 22:11:00                 1      15500      16500   

             updated_

In [23]:
# tính revenue
pns['revenue'] = pns['real_price'] * pns['quantity_ordered']

# tổng revenue theo từng sản phẩm và từng mức giá
revenue_by_product_and_price = pns.groupby(['product_id','real_price'])['revenue'].sum()
revenue_by_product_and_price

product_id  real_price
64          239000         956000
3954203     57500           57500
            60000          180000
            64000          640000
3998909     15500           15500
            16500          231000
            17000           34000
4085861     52000         1040000
            53500         2140000
            58000         2204000
            60000          180000
            62500         1812500
            67000          871000
Name: revenue, dtype: int64